<a href="https://colab.research.google.com/github/CanKeles5/ObjectDetection/blob/main/Stanford_Dataset_to_YOLO_Format_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2

In [3]:
#create the dataset, train, val folders
path_dataset = "/content/dataset"
path_train = path_dataset + "/train"
path_val = path_dataset + "/val"

os.mkdir(path_dataset)
os.mkdir(path_train)
os.mkdir(path_val)

In [20]:
img_width = 1983
img_height = 1088

In [4]:
dataset_size = 6000

In [5]:
split = 10

In [6]:
dataset_path = "/content/drive/MyDrive/data-yolov5"

In [ ]:
'''
for root, dirs, files in os.walk(dataset_path):
    for filename in files:
        print(filename)
'''

In [13]:
video_count = 0

For every video in videos, open the corresponding annotations file

In [14]:
for dirpath, dnames, fnames in os.walk(dataset_path + "/videos"):
    #print(str(dirpath) + " , " + str(dnames) + " , " + str(fnames))
    for f in fnames:
        if f == 'video.mov':
          video_count += 1

In [15]:
print("Number of videos is: " + str(video_count))

Number of videos is: 34


**dataset_size/video_count** is the number of frames we will extract from each video

In [10]:
num_frames = int(dataset_size/video_count)

Extract frames

In [23]:
data_count = 0

for dirpath, dnames, fnames in os.walk(dataset_path + "/videos"):
    #print(str(dirpath) + " , " + str(dnames) + " , " + str(fnames))
    for f in fnames:
        if f == 'video.mov':
          video_path = dirpath + "/video.mov"
          annot_path = dirpath.replace("videos", "annotations") + "/annotations.txt"

          print("Processing video: " + video_path)

          ##### Process the video #####
          vidcap = cv2.VideoCapture(video_path)
          success, image = vidcap.read()

          frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

          count = 0

          while success:
            if count % int(frame_count/num_frames) == 0:
              
              write_to = ""
              if data_count % split == 0:
                write_to = "/content/dataset/train"
              else:
                write_to = "/content/dataset/val"
              
              cv2.imwrite(write_to + "/frame" + str(data_count) + ".jpg", image)

              #### Create annotation
              annot_file = open(annot_path, "r")
              new_annot = open(write_to + "/label" + str(data_count) + ".txt", "x")

              #Read the annotation line by line
              while True:
                line = annot_file.readline()

                if not line:
                  break

                _, x_min, y_min, x_max, y_max, frameNo, lost, _, _, className = line.split()

                frameNo = int(frameNo)
                lost = int(lost)

                if frameNo==count:
                  x_min = float(x_min)
                  y_min = float(y_min)
                  x_max = float(x_max)
                  y_max = float(y_max)

                  className = className.replace('"', "")

                  classNo = -1

                  if className=="Biker":
                    classNo = 0
                  elif className=="Pedestrian":
                    classNo = 1
                  elif className=="Skater":
                    classNo = 2
                  elif className=="Cart":
                    classNo = 3
                  elif className=="Car":
                    classNo = 4
                  elif className=="Bus":
                    classNo = 5

                  if lost==0:
                    bbox_width = x_max - x_min
                    bbox_height = y_max - y_min

                    m_x = (bbox_width / 2 + x_min)/img_width
                    m_y = (bbox_height / 2 + y_min)/img_height

                    bbox_width = (x_max - x_min)/img_width
                    bbox_height = (y_max - y_min)/img_height

                    if data_count % split == 0:
                      label_file = open("/content/dataset/val/label" + str(data_count) + ".txt", "a")
                    else:
                      label_file = open("/content/dataset/train/label" + str(data_count) + ".txt", "a")

                    label_file.write(str(classNo) + " " + str(m_x) + " " + str(m_y) + " " + str(bbox_width) + " " + str(bbox_height) + "\n")

                    label_file.close()

              annot_file.close()

              data_count+=1

              '''
              if count % ratio == 0:
                write_to = path_val + "/frame" + str(count) + ".jpg"
              else:
                write_to = path_train + "/frame" + str(count) + ".jpg"
              
              cv2.imwrite(write_to, image)
              #cv2.imwrite(path_images + "/frame%d.jpg" % count, image)     # save frame as JPEG file
              '''
              '''
              #print('Read a new frame: ', success)
              '''
            
            success, image = vidcap.read()
            count += 1
          
          #############################


Processing video: /content/drive/MyDrive/data-yolov5/videos/coupa/video3/video.mov


KeyboardInterrupt: ignored

Create annotations for each frame